In [8]:
from symspellpy import SymSpell, Verbosity
from transformers import pipeline
import re

c:\Users\Asus\miniconda3\envs\yolo-env1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from collections import defaultdict
from wordfreq import top_n_list
english_vocab = set(top_n_list("en", 50000))

In [10]:
print('fertilized' in english_vocab)

True


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm

In [12]:
sym_spell = SymSpell()
sym_spell.load_pickle("../data/dictionary/symspell_dictionary.pkl")

True

In [13]:
# corpora = [
#     "../data/dictionary/dataset_wot_uncased_blanklines/processed_uncased_blanklines/wiki.txt",
#     "../data/dictionary/dataset_wot_uncased_blanklines/processed_uncased_blanklines/kompas.txt",
#     "../data/dictionary/dataset_wot_uncased_blanklines/processed_uncased_blanklines/tempo.txt",
#     "../data/dictionary/dataset_wot_uncased_blanklines/processed_uncased_blanklines/bppt.txt"
# ]

# for corpus in corpora:
#     sym_spell.create_dictionary(corpus)

# if "jumal" in sym_spell.words:
#     del sym_spell.words["jumal"]

In [14]:
# addon_path = "../data/dictionary/addon.txt"

# with open(addon_path, "r", encoding="utf-8") as f:
#     words = [w.strip() for w in f.readlines() if w.strip()]

# for word in words:
#     entry = sym_spell.words.get(word)
#     current_count = entry if entry is not None else 0
#     if current_count < 10:
#         sym_spell.create_dictionary_entry(word, 10 - current_count)

# sym_spell.save_pickle("../data/dictionary/symspell_dictionary.pkl")

In [15]:
# suggestions = sym_spell.lookup('ikbti', Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
# for s in suggestions:
#     print(s.term, s.count)

In [16]:
with open("../data/raw/ocr_result/ocr_1.txt", 'r', encoding="utf-8", errors="ignore") as file:
    text = file.read()
print(text)

Jumal Kecerdasan Buatan dan Teknologi Informasi (IKBTI) eISSN : 2964-2922
Vol. 2, No. 1, Januari 2023, hlm. 1-11 p-ISSN : 2963-6191

SISTEM PENDUKUNG KEPUTUSAN PENENTUAN ALAT KONTRASEPSI DI
BKKBN KABUPATEN CIREBON BERBASIS WEB MENGGUNAKAN
METODE MABAC

Rizki Purwowicaksono!, Faisal Akbar, Mohammad Rezza Fahlevvi*

Teknik Informatika, Sekolah Tinggi Ilmu Komputer Poltek Cirebon, Indonesia
3*Teknologi Rekayasa Informasi Pemerintahan, Fakultas Manajemen Pemerintahan_
Institut Pemerintahan Dalam Negeri, Indonesia

E-mail: 'purwowicaksonorizky@gmail.com, faisal.akbar@stikompoltek.ac.id, **rezza@ipdn.ac.id
(diterima : 12 Desember 2022, direvisi : 29 Desember 2022, diterima : 2 Januari 2023, dipublikasikan : 20 Januari 2023)

Abstrak

Keluarga Berencana merupakan program Pemerintah untuk menurunkan tingginya jumlah kelahiran penduduk,
dengan tidak diiringi peningkatan kualitas penduduk. Penggunaan alat kontrasepsi berguna untuk mencegah sel telur
dan sel sperma bertemu, menghentikan penggabun

In [17]:
ner = pipeline("ner", model="cahya/NusaBert-ner-v1.3", grouped_entities=True)
model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Device set to use cuda:0
c:\Users\Asus\miniconda3\envs\yolo-env1\Lib\site-packages\transformers\pipelines\token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(
Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
def find_typos(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    typos = defaultdict(list)
    
    for sentence in sentences:
        words = re.findall(r"\b[a-zA-Z]+\b", sentence.lower())
        
        for word in words:
            if word in english_vocab or len(word) <= 2:
                continue

            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
            best = suggestions[0]
            
            if (best.term != word and best.distance > 0) or (best.count < 10):
                if sentence.strip() not in typos[word]:  
                    typos[word].append(sentence.strip())

    return typos

In [19]:
def should_correct(entity_label, suggestion):
    if entity_label in {"PERSON", "GPE", "LOC"}:
        return False

    if entity_label == "ORG":
        if suggestion in sym_spell.words:
            return True
        return False
    
    if entity_label is None:
        return suggestion in sym_spell.words

    return False

In [20]:
ocr_confusions = {
    'rn': ['m'],
    'm': ['rn'],
    'l': ['t', 'i'],
    't': ['l'],
    '0': ['o'],
    '1': ['l', 'i'],
    'o': ['0'],
    'n': ['ri', 'ni'],
    'vv': ['w'],
    'w': ['vv'],
}

def expand_ocr_variants(word):
    variants = set()
    for pattern, subs in ocr_confusions.items():
        if pattern in word:
            for s in subs:
                variants.add(re.sub(pattern, s, word))
    return variants

In [21]:
def suggest_words(word):
    suggest_symspell = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
    
    valid_suggestions = [s for s in suggest_symspell if s.count > 10]
    if valid_suggestions: best = valid_suggestions[0].term
    else: best = suggest_symspell[0].term

    variants = expand_ocr_variants(word)

    final_s = set()
    final_s.add(best)

    for var in variants:
        if var in sym_spell.words:
            final_s.add(var)
    return final_s

In [22]:
def output_highest(model, tokenizer, sentence, cands, word):
    sentence = sentence.lower()
    sentence = sentence.replace(word,'[MASK]')

    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    mask_token_id = tokenizer.mask_token_id
    mask_token_index = (inputs.input_ids == mask_token_id).nonzero(as_tuple=True)[1]
    mask_logits = logits[0, mask_token_index, :]

    candidate_ids = tokenizer.convert_tokens_to_ids(cands)
    candidate_scores = mask_logits[0, candidate_ids]

    # print("--- Hasil Perbandingan ---")
    # for i, candidate in enumerate(cands):
    #     print(f"  - {candidate:<10}: {candidate_scores[i].item():.4f}")
    
    best_candidate_index = candidate_scores.argmax().item()
    best_candidate = cands[best_candidate_index]
    return best_candidate

In [23]:
def context_correct(typos):
    corrections = []

    for word, sentences in tqdm(typos.items(), desc='Processing words'):
        for sentence in sentences:
            doc = ner(sentence)
            entity_label = None

            # cari entity label kata typo
            for ent in doc:
                if word.lower() in ent["word"].lower():
                    entity_label = ent["entity_group"]
                    break
            
            # apakah word blh dicorrect?
            # if(not should_correct(entity_label, word)): continue
            
            suggestions = suggest_words(word)
            if not suggestions:
                continue #kalau gada suggestions

            best_word = output_highest(model, tokenizer, sentence, list(suggestions), word)
            if(not should_correct(entity_label, best_word)): continue
            corrections.append({"word":word, "correction":best_word})
            
    return corrections

In [24]:
typos = find_typos(text)
typos

defaultdict(list,
            {'jumal': ['Jumal Kecerdasan Buatan dan Teknologi Informasi (IKBTI) eISSN : 2964-2922\nVol.'],
             'mabac': ["1-11 p-ISSN : 2963-6191\n\nSISTEM PENDUKUNG KEPUTUSAN PENENTUAN ALAT KONTRASEPSI DI\nBKKBN KABUPATEN CIREBON BERBASIS WEB MENGGUNAKAN\nMETODE MABAC\n\nRizki Purwowicaksono!, Faisal Akbar, Mohammad Rezza Fahlevvi*\n\nTeknik Informatika, Sekolah Tinggi Ilmu Komputer Poltek Cirebon, Indonesia\n3*Teknologi Rekayasa Informasi Pemerintahan, Fakultas Manajemen Pemerintahan_\nInstitut Pemerintahan Dalam Negeri, Indonesia\n\nE-mail: 'purwowicaksonorizky@gmail.com, faisal.akbar@stikompoltek.ac.id, **rezza@ipdn.ac.id\n(diterima : 12 Desember 2022, direvisi : 29 Desember 2022, diterima : 2 Januari 2023, dipublikasikan : 20 Januari 2023)\n\nAbstrak\n\nKeluarga Berencana merupakan program Pemerintah untuk menurunkan tingginya jumlah kelahiran penduduk,\ndengan tidak diiringi peningkatan kualitas penduduk.",
              'Pemilihan alat kontrasepsi terb

In [25]:
res = context_correct(typos)
for obj in res:
    print(f"{obj['word']} --> {obj['correction']}")

Processing words: 100%|██████████| 10/10 [00:04<00:00,  2.20it/s]

jumal --> jurnal
inputan --> inputan
perangkingan --> perangkingan
perangkingan --> perangkingan
